In [2]:
import pandas as pd
import datetime

In [3]:
channel = "Scikitlearn"
channel2 = "Angular"

## Read users

In [2]:
df = pd.read_json('../Gitter_Channels/Scikitlearn/Scikitearn_user.json')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4318 entries, 0 to 4317
Data columns (total 3 columns):
displayName    4318 non-null object
id             4318 non-null object
username       4318 non-null object
dtypes: object(3)
memory usage: 134.9+ KB


In [4]:
df.head()

,displayName,id,username
0,Jitesh Khandelwal,529c6a18ed5ab0b3bf04d26a,jiteshk23
1,Rahul Kulhari,529c6bdded5ab0b3bf04d758,RahulKulhari
2,Jaepil Jeong,529c6c0bed5ab0b3bf04d7db,jaepil
3,Edwin Khoo,529c6c8eed5ab0b3bf04d94a,edwinksl
4,Thanh Nguyen,529c6c9bed5ab0b3bf04d96b,tienthanh8490


## Read chat logs

In [4]:
import json

In [5]:
save_path = "../../Analysis/"

In [6]:
def read_logs(utterance_list):
    prev_user = ""
    dialog_id = 0
    results = []
    profile = {}
    isHead =False
    create_time = ""
    line = 0
    
    for utterance in utterance_list:     
        if len(utterance) == 0:
            continue
        if '-------------------------------------' in utterance:
            if dialog_id != 0:
                issue["Id"] = dialog_id
                #print(issue["Id"])
                issue["asker"] = prev_user
                #print(issue["asker"])
                issue["time"] = create_time
                #print(issue["time"])
                issue["content"] = content
                #print(issue["content"])
                issue["lines"] = line
                #print(issue["lines"])
                results.append(issue)
            dialog_id += 1
            line = 0
            issue = {}
            content = ""
            isHead =True
        else:
            line += 1
            start_index = 0
            end_index = 0
            #retreive time
            while utterance[start_index] != "[":
                start_index += 1
        
            start_index += 1
            end_index = start_index
            while utterance[end_index] != "]":
                end_index += 1
        
            time = utterance[start_index:end_index]
        
            #retreive user
            start_index = end_index
            while utterance[start_index] != "<":
                start_index += 1
        
            start_index += 1
            end_index = start_index
            while utterance[end_index] != ">":
                end_index += 1
            
            curr_user= utterance[start_index:end_index]
        
            #retreive message
            message = utterance[(end_index+2):]
            #print("message:",message)

            
            if line == 1:
                prev_user = curr_user
                create_time = time
            
            if isHead and prev_user == curr_user:
                content += message
            else:
                isHead =False
                if prev_user == curr_user:
                    continue
            
                if len(profile) > 0 and curr_user in profile.keys():
                    if dialog_id in profile[curr_user].keys():
                        profile[curr_user][dialog_id]["Answer"] += message
                    else:
                        profile[curr_user][dialog_id] = {}
                        profile[curr_user][dialog_id]["Time"] = time
                        profile[curr_user][dialog_id]["Answer"] = message
    
                else:
                    profile[curr_user] = {}
                    profile[curr_user][dialog_id] = {}
                    profile[curr_user][dialog_id]["Time"] = time
                    profile[curr_user][dialog_id]["Answer"] = message
    
    return results, profile

### Scikitlearn

In [93]:
with open("../Gitter_Channels/"+channel+"/disentangle/"+channel+"_result.txt", "r") as f:
        utterance_list = f.readlines()

In [ ]:
scikitlearn_results,scikitlearn_profile = read_logs(utterance_list)

In [91]:
len(scikitlearn_results)

5414

In [ ]:
with open(save_path+channel+'/Scikitlearn_issue.json', 'w') as outfile:
    json.dump(scikitlearn_results, outfile)
with open(save_path+channel+'/Scikitlearn_profile.json', 'w') as outfile:
    json.dump(scikitlearn_profile, outfile)

### Angular

In [ ]:
with open("../Gitter_Channels/"+channel2+"/disentangle/"+channel2+"_result.txt", "r") as f:
        utterance_list = f.readlines()

In [ ]:
augular_results,angluar_profile = read_logs(utterance_list)

In [ ]:
len(augular_results)

In [ ]:
with open(save_path+channel2+'/Angular_issue.json', 'w') as outfile:
    json.dump(augular_results, outfile)
with open(save_path+channel2+'/Angular_profile.json', 'w') as outfile:
    json.dump(angluar_profile, outfile)

## Convert to csv

In [7]:
def strToDatetime(str):
    strptime = datetime.datetime.strptime(str, "%Y-%m-%d %H:%M")
    return strptime

In [8]:
def conv_to_dateframe(profile):
    user_Id = []
    cluster_Id = []
    answer_date = []
    answers = []

    for user in profile:
        for cluster in profile[user]:
            #print(profile[user][cluster]["Time"])
            time = strToDatetime(profile[user][cluster]["Time"])
            user_Id.append(user)
            cluster_Id.append(cluster)
            answer_date.append(time)
            answers.append(profile[user][cluster]["Answer"])
    
    df_user = pd.DataFrame()
    df_user["user_Id"] =user_Id
    df_user["cluster_Id"] =cluster_Id
    df_user["answer_date"] =answer_date
    df_user["answers"] =answers
    
    return df_user

### Scikitlearn

In [99]:
with open(save_path+channel+'/'+channel+'_profile.json', 'r') as f:
    profile = json.load(f)

In [104]:
len(profile)

106

In [105]:
df_scikitlearn_profile = conv_to_dateframe(profile)

In [107]:
df_scikitlearn_profile.head()

,user_Id,cluster_Id,answer_date,answers
0,54d4a1d6db8155e6700f853b,12,2015-02-06 11:14:00,I'm with family this week and so not super act...
1,54d4a1d6db8155e6700f853b,61,2015-02-24 22:46:00,cool\nok\n
2,54d4a1d6db8155e6700f853b,69,2015-02-25 18:59:00,yeah that sounds like a good idea\n
3,54d4a1d6db8155e6700f853b,72,2015-02-25 19:14:00,you mean == 1\n
4,54d4a1d6db8155e6700f853b,75,2015-02-25 19:19:00,that is also weird ;)\n


In [108]:
df_scikitlearn_profile.to_csv(save_path+channel+'/'+channel+'_profile.csv', index=False) 

### Angular

In [9]:
with open(save_path+channel2+'/'+channel2+'_profile.json', 'r') as f:
    profile = json.load(f)

In [10]:
len(profile)

6707

In [11]:
df_angular_profile = conv_to_dateframe(profile)

In [12]:
df_angular_profile.head()

,user_Id,cluster_Id,answer_date,answers
0,54e6ee3b15522ed4b3dc3c24,15,2015-03-08 18:01:00,currently only working in chrome until shadow ...
1,54e6ee3b15522ed4b3dc3c24,20,2015-03-09 02:39:00,::shadow\nhttp://www.html5rocks.com/en/tutoria...
2,54e6ee3b15522ed4b3dc3c24,100,2015-03-13 19:48:00,@mhevery that was my worry too (Dart version ...
3,54e6ee3b15522ed4b3dc3c24,113,2015-03-14 02:46:00,you donat need to inject it since itas going t...
4,54e6ee3b15522ed4b3dc3c24,118,2015-03-14 07:23:00,what version of node?\n


In [ ]:
df_angular_profile.to_csv(save_path+channel2+'/'+channel2+'_profile.csv', index=False) 